In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [2]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

/Users/mikenlee/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (2.0.3) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Prepare ETL for the Class Participant data

In [7]:
#class df as a parent to staff assignment df
classparticipant_df = pd.read_sql("SELECT * FROM classparticipant", conn)
classparticipant_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


In [8]:
#rename columns to match salesforce
classparticipant_df.rename(columns={
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c'
}, inplace=True)

classparticipant_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate__c,EndDate__c
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


In [139]:
#select relevant columns
# staffassignment_df = staffassignment_df[['ID_Staff', 'ID_Class', 'Role__c', 'StartDate__c', 'EndDate__c']]
# staffassignment_df.head()

,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c
0,1,1,Teacher Assistant,2020-09-16,None
1,4,1,Teacher Assistant,2020-09-16,None
2,3,1,Instructor,2020-09-16,None


In [10]:
#format date columns
classparticipant_df['StartDate__c'] = pd.to_datetime(classparticipant_df['StartDate__c']).dt.date
classparticipant_df['EndDate__c'] = None
classparticipant_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate__c,EndDate__c
0,1,33,1,2020-09-16,None
1,2,34,1,2020-09-16,None
2,3,35,1,2020-09-16,None
3,4,62,1,2020-09-16,None
4,5,36,1,2020-09-16,None


# Create Class Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Class` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [11]:
#extract class object from salesforce
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Section__c, Start_Date__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'Section__c': row['Section__c'],
        'Start_Date__c': row['Start_Date__c']
    }
    class_lookup_list.append(rec)
    
class_lookup_list

[{'ID_Class__c': 'a014x000008VFla',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': '2019-05-14'},
 {'ID_Class__c': 'a014x000008VFlV',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': '2018-05-15'},
 {'ID_Class__c': 'a014x000008VFlQ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': '2020-09-16'},
 {'ID_Class__c': 'a014x000008VFkd',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': '2019-05-14'}]

In [12]:
#convert to DF
class_lookup_list = pd.DataFrame(class_lookup_list)
class_lookup_list

,ID_Class__c,Section__c,Start_Date__c
0,a014x000008VFla,GWARL201905UIUX3,2019-05-14
1,a014x000008VFlV,GWDC201805DATA3,2018-05-15
2,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
3,a014x000008VFkd,GWARL201905WEB3,2019-05-14


### Query original 'Class' table from MySQL
Join with class_lookup_list on 'Section' column
then join with staff assignment on the original ID_class

In [13]:
query = '''
    SELECT *
    FROM 
        class 
'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [14]:
#leave out 'ID_Course' column since it isn't in the staffassignment object
class_data_df = class_data_df[['ID_Class', 'Section']]
class_data_df

,ID_Class,Section
0,1,GWU-ARL-DATA-PT-09-0
1,2,GWDC201805DATA3
2,3,GWARL201905WEB3
3,4,GWARL201905UIUX3


In [15]:
#rename columns
class_data_df.rename(columns={
    'Section':'Section__c'
}, inplace=True)

class_data_df

,ID_Class,Section__c
0,1,GWU-ARL-DATA-PT-09-0
1,2,GWDC201805DATA3
2,3,GWARL201905WEB3
3,4,GWARL201905UIUX3


### Join the class DataFrame with the Class object from SF
This join is necessary to get a DF with the original ID_Class and the SF 'name'/'ID_Class__c'

In [16]:
class_lookup_df = pd.merge(class_lookup_list, class_data_df, how='left')

class_lookup_df

,ID_Class__c,Section__c,Start_Date__c,ID_Class
0,a014x000008VFla,GWARL201905UIUX3,2019-05-14,4
1,a014x000008VFlV,GWDC201805DATA3,2018-05-15,2
2,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,1
3,a014x000008VFkd,GWARL201905WEB3,2019-05-14,3


# Prepare ETL for the `Student` data
push up the salesforce to get `Name` and pull back to join

In [30]:
# Staff DF from AWS MySQL
student_df = pd.read_sql("SELECT * FROM student", conn)
student_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [31]:
#rename columns to match salesforce
student_df.rename(columns={
    'StudentID':'StudentID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate':'BirthDate__c',
    'Gender':'Gender__c'
}, inplace=True)

student_df.head()

,ID_Student,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [32]:
#select relevant columns
student_df.drop(columns = ['ID_Student'], inplace=True)
student_df.head()

,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [33]:
#format date columns
student_df['BirthDate__c'] = None

student_df.head()

,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [ ]:
#convert to dictionary
student_data_records = student_df.to_dict(orient='records')
student_data_records

### Insert `Student` records into SalesForce

In [36]:
for rec in student_data_records:
 
    record = {
        'StudentID__c': rec['StudentID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': rec['BirthDate__c'],
        'Gender__c': rec['Gender__c'],
    }
        
    try:
        sf.Student__C.create(record)
    except Exception as e:
        print(e)

## Pull `student` from SF and join with Staff from AWS to join with `student` from Salesforce 
This is to get the original `student` ID column and to join with the SalesForce `student` "Name"

In [ ]:
student_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT StudentID__c, Name FROM Student__c")
for row in data:
    rec = {
        'ID_student__c': row['Name'], # this is a critical line of code
        'StudentID__c': row['StudentID__c']
    }
    student_lookup_list.append(rec)
    
student_lookup_list

In [40]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head()

,ID_student__c,StudentID__c
0,a044x000002rroG,25006348
1,a044x000002rroQ,25005250
2,a044x000002rrnr,25005602
3,a044x000002rrmo,25004961
4,a044x000002rrnS,25003605


In [42]:
# Staff DF from AWS MySQL
student_aws = pd.read_sql("SELECT StudentID, ID_Student FROM student", conn)
student_aws.head()

,StudentID,ID_Student
0,25004961,33
1,25003514,34
2,25005833,35
3,25002589,36
4,25007185,37


In [44]:
student_aws.rename(columns = {
    'StudentID':'StudentID__c'
}, inplace = True)

student_aws

,StudentID__c,ID_Student
0,25004961,33
1,25003514,34
2,25005833,35
3,25002589,36
4,25007185,37
5,25006014,38
6,25007528,39
7,25003778,40
8,25003605,41
9,25002056,42


In [46]:
student_junction = pd.merge(student_lookup_df, student_aws, how = 'left')
# student_junction = student_junction[['ID_staff__c', 'ID_Staff']]
student_junction.head()

,ID_student__c,StudentID__c,ID_Student
0,a044x000002rroG,25006348,51
1,a044x000002rroQ,25005250,53
2,a044x000002rrnr,25005602,46
3,a044x000002rrmo,25004961,33
4,a044x000002rrnS,25003605,41


# Join `ClassParticipant` DataFrame with `Class` lookup table and `Student` junction table

In [47]:
#merge with Class look up to get the `Name` column added
classparticipant_df = pd.merge(classparticipant_df, class_lookup_df, how='left')
classparticipant_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate__c,EndDate__c,ID_Class__c,Section__c,Start_Date__c
0,1,33,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
1,2,34,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
2,3,35,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
3,4,62,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
4,5,36,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16


In [48]:
#merge with Staff junction to get the `Name` column added
classparticipant_df = pd.merge(classparticipant_df, student_junction, how='left')
classparticipant_df.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate__c,EndDate__c,ID_Class__c,Section__c,Start_Date__c,ID_student__c,StudentID__c
0,1,33,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a044x000002rrmo,25004961
1,2,34,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a044x000002rrmt,25003514
2,3,35,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a044x000002rrmy,25005833
3,4,62,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a044x000002rrp9,25007334
4,5,36,1,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a044x000002rrn3,25002589


In [49]:
classparticipant_df = classparticipant_df[['ID_Class__c', 'ID_student__c', 'StartDate__c', 'EndDate__c']]
classparticipant_df.head()

,ID_Class__c,ID_student__c,StartDate__c,EndDate__c
0,a014x000008VFlQ,a044x000002rrmo,2020-09-16,None
1,a014x000008VFlQ,a044x000002rrmt,2020-09-16,None
2,a014x000008VFlQ,a044x000002rrmy,2020-09-16,None
3,a014x000008VFlQ,a044x000002rrp9,2020-09-16,None
4,a014x000008VFlQ,a044x000002rrn3,2020-09-16,None


In [ ]:
#convert to dictionary
classparticipant_records = classparticipant_df.to_dict('records')
classparticipant_records

## Insert `classparticipant` Records into SalesForce

In [51]:
#load into salesforce
for rec in classparticipant_records:

    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_student__c': rec['ID_student__c'],
        'EndDate__c': rec['EndDate__c'],
        'StartDate__c': str(rec['StartDate__c'])
    }
    
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
sf.bulk.Staff_Assignment__c.insert(staffassignment_records)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [149]:
staffassignment_records = sf.query("SELECT Id FROM Staff_Assignment__c")
recs_to_delete = [{'Id': r['Id']} for r in staffassignment_records['records']]
recs_to_delete

[{'Id': 'a024x000002hKgCAAU'},
 {'Id': 'a024x000002hKgHAAU'},
 {'Id': 'a024x000002hKgMAAU'}]

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [150]:
for rec in recs_to_delete:
    try:
        sf.Staff_Assignment__c.delete(rec['Id'])
    except Exception as e:
        print(e)